# What's new in Brightway 2.5

## Backwards compatibility

Compatilibility with Brightway 2 has been maintained whenever possible, but there are a few cases where compatiblity could not be kept.

* In `bw2calc`, the `LCA` class now takes over responsibility for all types of LCA calculations, including Monte Carlo.
* In `bw2data`, `Database.get()` and `Database().get()` are no longer supported. Use `get_node(database="something", **other_filters)` instead.

In [1]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi

In [2]:
# if "2.5 examples in action" in bd.projects:
#     bd.projects.delete_project("2.5 examples in action", True)
bd.projects.set_current("2.5 examples in action")

In [3]:
bi.add_example_database()

/Users/cmutel/miniconda3/envs/bw25dev/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
Writing activities to SQLite3 database:


Extracted 4 worksheets in 0.02 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: assign_only_product_as_production
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 11 strategies in 0.00 seconds
Applying strategy: link_iterable_by_fields
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


0% [#########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/27/2022 23:00:40
  Finished: 09/27/2022 23:00:40
  Total time elapsed: 00:00:00
  CPU %: 107.10
  Memory %: 0.52
Created database: Mobility example


In [4]:
bi.bw2setup()

Writing activities to SQLite3 database:


Creating default biosphere

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.00 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/27/2022 23:00:47
  Finished: 09/27/2022 23:00:47
  Total time elapsed: 00:00:00
  CPU %: 102.40
  Memory %: 0.64
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: fix_ecoinvent_38_lcia_implementation
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 5 strategies in 0.61 seconds
Wrote 975 LCIA methods with 254388 characterization factors
Creating core data migrations



# `bw2data`

`Make steel` -> `Activity`

`Drive motorcycle` -> `Activity`

`Grow sugarbeet` -> `Activity`

`CO2` -> `Activity`????

## Change in preferred nomenclature

`Activity` -> `Node`

`Exchange` -> `Edge`

In [7]:
steel = bd.get_activity(('Mobility example', 'Steel'))

In [8]:
isinstance(steel, bd.Node)

True

In [ ]:
steel.edges(), steel.new_edge(), db.new_node()

## OMGWTFBBQ

In [9]:
[act 
 for act in bd.Database('Mobility example')
 if act['name'] == 'Steel'
][0]

'Steel' (kilogram, GLO, None)

In [10]:
bd.get_node(database='Mobility example', code='Steel')

'Steel' (kilogram, GLO, None)

## Any attribute

In [11]:
steel = bd.get_node(name='Steel')
steel['foo'] = 'bar'
steel.save()

In [12]:
bd.get_node(foo='bar')

/Users/cmutel/Code/bw2/data/bw2data/utils.py:429: UserWarning: Given search criteria very broad; try to specify at least a database
  warnings.warn(


'Steel' (kilogram, GLO, None)

## `get_node` returns only one node

`bw2data.errors.UnknownObject` if no nodes can be found.

`bw2data.errors.MultipleResults` if more than one node.

## `get_id` and the removal of `mapping`

No more `mapping.pickle` file. `id` comes from the database and is preferred.

In [13]:
steel.id

4

In [14]:
bd.get_id(steel)

4

## Let's write SQL

`Node` table has columns for `code`, `database`, `location`, `name`, `product`, `type`.

In [15]:
from bw2data.backends import ActivityDataset as AD
AD.update(name="Wow, this is some car!").where(AD.name == 'Combustion car').execute()

1

In [16]:
bd.get_node(name="Wow, this is some car!")

'Wow, this is some car!' (unit, GLO, ('Combustion car',))

## `Activity` attribute lookups

Make the world more magical. Boo:

In [17]:
steel = bd.get_node(name='Steel')
steel['properties'] = {'carbon content': {'amount': 0.01}}
steel['classifications'] = {'ISIC': {'code': '2410', 'system': 'ISIC Rev. 4'}}
steel.save()

In [18]:
[value for key, value in steel['properties'].items() if key == 'carbon content']

[{'amount': 0.01}]

Yeah!

In [19]:
steel['carbon content']

{'amount': 0.01}

In [20]:
steel['ISIC']

{'code': '2410', 'system': 'ISIC Rev. 4'}

## Reference products

No!!

In [21]:
[exc for exc in steel.exchanges() if exc['type'] == 'production'][0]

Exchange: 1 kilogram 'Steel' (kilogram, GLO, None) to 'Steel' (kilogram, GLO, None)>

Yes!!

In [22]:
steel.rp_exchange()

Exchange: 1 kilogram 'Steel' (kilogram, GLO, None) to 'Steel' (kilogram, GLO, None)>

## Properties of reference products

Double yes!!

In [23]:
exc = steel.rp_exchange()
exc['properties'] = {'iron content': 0.98}
exc.save()

In [24]:
steel['iron content']

0.98

## Easier data cleanup

If you are building inventories manually, it is easy to accidentally add an exchange too many times:

In [25]:
steel, co2 = bd.get_node(name="Steel"), bd.get_node(name="CO2")

for _ in range(5):
    steel.new_edge(input=co2, amount=1.5, type="biosphere").save()

We can now easily clean up these duplicates:

In [26]:
bd.Database('Mobility example').delete_duplicate_exchanges()

Deleting exchange: Exchange: 1.5 kilogram 'CO2' (kilogram, None, None) to 'Steel' (kilogram, GLO, None)>
Deleting exchange: Exchange: 1.5 kilogram 'CO2' (kilogram, None, None) to 'Steel' (kilogram, GLO, None)>
Deleting exchange: Exchange: 1.5 kilogram 'CO2' (kilogram, None, None) to 'Steel' (kilogram, GLO, None)>
Deleting exchange: Exchange: 1.5 kilogram 'CO2' (kilogram, None, None) to 'Steel' (kilogram, GLO, None)>
Deleting exchange: Exchange: 1.5 kilogram 'CO2' (kilogram, None, None) to 'Steel' (kilogram, GLO, None)>


In [27]:
for exc in steel.biosphere():
    print(exc)

Exchange: 1.5 kilogram 'CO2' (kilogram, None, None) to 'Steel' (kilogram, GLO, None)>


## Filepaths are instances of `pathlib.Path`

`Path` objects are [pretty great](https://treyhunner.com/2018/12/why-you-should-be-using-pathlib/), you should [use them](https://docs.python.org/3/library/pathlib.html).

In [28]:
type(bd.projects.dir), type(bd.projects.logs_dir)

(pathlib.PosixPath, pathlib.PosixPath)

In [29]:
mobility = bd.Database('Mobility example')

In [30]:
mobility.dirpath_processed() / "filename.zip"

PosixPath('/Users/cmutel/Library/Application Support/Brightway3/25-examples-in-action.93bc6058/processed/filename.zip')

## Easier access to `Datapackages`

This guy is dumb!

In [31]:
from fs.zipfs import ZipFS
import bw_processing as bwp

bwp.load_datapackage(ZipFS(mobility.filepath_processed()))

That's better...

In [32]:
mobility.datapackage()

In [33]:
bi.exiobase_monetary()

764936192it [02:29, 5126539.45it/s]                                 
Writing activities to SQLite3 database:


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/27/2022 22:57:16
  Finished: 09/27/2022 22:57:16
  Total time elapsed: 00:00:00
  CPU %: 101.90
  Memory %: 0.60


Writing activities to SQLite3 database:


Created new database for EXIOBASE-specific biosphere flows: EXIOBASE 3.8.1 2017 monetary biosphere


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/27/2022 22:57:17
  Finished: 09/27/2022 22:57:17
  Total time elapsed: 00:00:00
  CPU %: 101.00
  Memory %: 0.62
Created database of EXIOBASE activity metadata
Patched 0 LCIA methods with unit 'kg CO2-Eq'
Patching LCIA methods with EXIOBASE flows


NameError: name 'IOTableBackend' is not defined

## IOTable improvements

This is being rewritten in the `file-remover-progressive` branch.

## Brightway ❤️ Pandas

In [30]:
bi.useeio11()

Unzipping file
Importing data
Applying strategy: json_ld_allocate_datasets
Applying strategy: json_ld_get_normalized_exchange_locations
Applying strategy: json_ld_convert_unit_to_reference_unit
Applying strategy: json_ld_get_activities_list_from_rawdata
Applying strategy: json_ld_add_products_as_activities
Applying strategy: json_ld_get_normalized_exchange_units
Applying strategy: json_ld_add_activity_unit
Applying strategy: json_ld_rename_metadata_fields
Applying strategy: json_ld_location_name
Applying strategy: json_ld_remove_fields
Applying strategy: json_ld_fix_process_type
Applying strategy: json_ld_label_exchange_type
Applying strategy: json_ld_prepare_exchange_fields_for_linking
Applying strategy: add_database_name
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applied 17 strategies in 0.57 seconds
Moved 1873 biosphere flows to `self.data`
2649 datasets
162926 exchanges
0 unlinked exchanges
  
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Title: Writing activities to SQLite3 database:
  Started: 09/27/2022 21:22:59
  Finished: 09/27/2022 21:23:03
  Total time elapsed: 00:00:04
  CPU %: 98.30
  Memory %: 1.95
Created database: US EEIO 1.1
Applying strategy: json_ld_lcia_add_method_metadata
Applying strategy: json_ld_lcia_convert_to_list
Applying strategy: json_ld_lcia_set_method_metadata
Applying strategy: json_ld_lcia_reformat_cfs_as_exchanges
Applying strategy: normalize_units
Applied 5 strategies in 0.00 seconds
19 methods
4511 cfs
0 unlinked cfs
Wrote 19 LCIA methods with 4511 characterization factors


Pick a product and an activity node at random.

In [31]:
product = next(node for node in bd.Database("US EEIO 1.1") if node['type'] == 'product')
activity = next(node for node in bd.Database("US EEIO 1.1") if node['type'] == 'process')
product, activity

('Sugar, candy, and chocolate; at manufacturer' (, United States, ('31-33: Manufacturing', '3113: Sugar and Confectionery Product Manufacturing')),
 'General merchandise stores' (USD, United States, None))

The first dataframe is all the nodes (processes or activities) in the given database:

In [32]:
df = bd.Database("US EEIO 1.1").nodes_to_dataframe()
df

,CAS number,categories,classifications,code,database,description,dqEntry,dqSystem,exchangeDqSystem,filename,id,location,modified,name,processDocumentation,type,unit,version
497,,"(water, unspecified)",NaN,2ee4697d-b7f4-362b-86a4-94b644699500,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,1143,NaN,NaN,"(2,4-DICHLOROPHENOXY)ACETIC ACID COMPD. WITH 2...",NaN,emission,,NaN
623,,"(soil, groundwater)",NaN,5b98f875-8d1c-3549-a7df-28d7d90e7ccb,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,1436,NaN,NaN,"(2,4-DICHLOROPHENOXY)ACETIC ACID COMPD. WITH 2...",NaN,emission,,NaN
718,,"(air, low population density)",NaN,6ca23b5d-83dc-3b02-bf39-8eabf9d41151,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,1649,NaN,NaN,"(2,4-DICHLOROPHENOXY)ACETIC ACID COMPD. WITH 2...",NaN,emission,,NaN
107,,"(air, low population density)",NaN,3404c9d4-8d41-36cb-8a95-c8b428518cfa,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,1175,NaN,NaN,(4-CHLORO-2-METHYLPHENOXY)ACETIC ACID COMPD. W...,NaN,emission,,NaN
841,,"(water, unspecified)",NaN,93086e32-c013-3e34-a074-4760c72fe775,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,1895,NaN,NaN,(4-CHLORO-2-METHYLPHENOXY)ACETIC ACID COMPD. W...,NaN,emission,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2120,,"(air, unspecified)",NaN,d9a5b786-d06c-44af-a088-b070aa605d9b,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,2379,NaN,NaN,trifluralin,NaN,emission,,NaN
572,7440622.0,"(water, unspecified)",NaN,63e8256e-8549-11e0-9d78-0800200c9a66,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,1586,NaN,NaN,vanadium,NaN,emission,,NaN
1374,7440622.0,"(soil, industrial)",NaN,1a5850a0-0069-4b73-bb91-7a61e8d45ae5,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,980,NaN,NaN,vanadium,NaN,emission,,NaN
2011,7440622.0,"(air, unspecified)",NaN,591b0a62-8064-4697-86ed-47bfa1f8b5e6,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,1416,NaN,NaN,vanadium,NaN,emission,,NaN


The columns come from the data attributes stored on the nodes. If one node has the attribute, it is added as a column. You can control which columns get returned, and how they are sorted, see the docstring.

This is a normal dataframe, so you can filter it, add or remove columns, and sort as desired.

In [33]:
df.columns

Index(['CAS number', 'categories', 'classifications', 'code', 'database',
       'description', 'dqEntry', 'dqSystem', 'exchangeDqSystem', 'filename',
       'id', 'location', 'modified', 'name', 'processDocumentation', 'type',
       'unit', 'version'],
      dtype='object')

We can also list all the edges (exchanges) as a dataframe. This is normally too much information, and can take a bit of time to produce, but can be useful.

In [34]:
df = bd.Database("US EEIO 1.1").edges_to_dataframe()
df

Getting activity data


100%|███████████████████████| 2649/2649 [00:00<00:00, 312160.01it/s]


Adding exchange data to activities


100%|████████████████████| 162926/162926 [00:02<00:00, 66576.72it/s]


Filling out exchange data


100%|███████████████████████| 2649/2649 [00:00<00:00, 152688.87it/s]


Creating DataFrame
Compressing DataFrame


,target_id,target_database,target_code,target_name,target_reference_product,target_location,target_unit,target_type,source_id,source_database,source_code,source_name,source_product,source_location,source_unit,source_categories,edge_amount,edge_type
0,10,US EEIO 1.1,01624075-b520-3826-bd73-2068f7aa24e7,Frozen food; at manufacturer,NaN,United States,USD,process,468,US EEIO 1.1,26bed504-3f97-3a2b-aa83-ffbe94f3b371,Frozen food; at manufacturer,NaN,United States,,NaN,1.000000e+00,production
1,10,US EEIO 1.1,01624075-b520-3826-bd73-2068f7aa24e7,Frozen food; at manufacturer,NaN,United States,USD,process,446,US EEIO 1.1,1bafcbbb-dbe0-338d-b9c1-8c355426cbef,State and local government enterprises,NaN,United States,,NaN,1.338032e-03,technosphere
2,10,US EEIO 1.1,01624075-b520-3826-bd73-2068f7aa24e7,Frozen food; at manufacturer,NaN,United States,USD,process,1029,US EEIO 1.1,20185046-64bb-4c09-a8e7-e8a9e144ca98,Dinitrogen monoxide,NaN,NaN,,NaN,1.114413e-07,biosphere
3,10,US EEIO 1.1,01624075-b520-3826-bd73-2068f7aa24e7,Frozen food; at manufacturer,NaN,United States,USD,process,1737,US EEIO 1.1,7ae398b3-8532-11e0-9d78-0800200c9a66,ethylene glycol,NaN,NaN,,NaN,1.223946e-07,biosphere
4,10,US EEIO 1.1,01624075-b520-3826-bd73-2068f7aa24e7,Frozen food; at manufacturer,NaN,United States,USD,process,417,US EEIO 1.1,0bb0108f-c486-32b3-b059-e0c6c8380571,"Tobacco, cotton, sugarcane, peanuts, sugar bee...",NaN,United States,,NaN,9.990870e-06,technosphere
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162921,397,US EEIO 1.1,fe5971a7-2610-32ca-8193-a94873de130c,Automatic controls for HVAC and refrigeration ...,NaN,United States,USD,process,1435,US EEIO 1.1,5b2a19b9-1243-44ae-b76c-c0d92159d5d6,2-METHOXYETHANOL,NaN,NaN,,NaN,8.602175e-12,biosphere
162922,397,US EEIO 1.1,fe5971a7-2610-32ca-8193-a94873de130c,Automatic controls for HVAC and refrigeration ...,NaN,United States,USD,process,2631,US EEIO 1.1,fd7aa71c-508c-480d-81a6-8052aad92646,sulfur dioxide,NaN,NaN,,NaN,2.635425e-08,biosphere
162923,397,US EEIO 1.1,fe5971a7-2610-32ca-8193-a94873de130c,Automatic controls for HVAC and refrigeration ...,NaN,United States,USD,process,1471,US EEIO 1.1,5fd672a0-cb68-39e6-88dc-db1a9281c57b,"2,4,5,2',5'-PENTACHLOROBIPHENYL",NaN,NaN,,NaN,9.791193e-19,biosphere
162924,397,US EEIO 1.1,fe5971a7-2610-32ca-8193-a94873de130c,Automatic controls for HVAC and refrigeration ...,NaN,United States,USD,process,598,US EEIO 1.1,7e5c6ee1-a47e-3afd-9278-dcb5e1ee50b5,"Screws, nuts, and bolts; at manufacturer",NaN,United States,,NaN,1.810643e-02,technosphere


Now we have standard column labels. as these are directed edges, they have a source and a target. Most of the columns should be self-explanatory. Note that we differentiate between `'target_reference_product'` and `'source_product'`, and only provide the `categories` on the `source`.

In [35]:
df.columns

Index(['target_id', 'target_database', 'target_code', 'target_name',
       'target_reference_product', 'target_location', 'target_unit',
       'target_type', 'source_id', 'source_database', 'source_code',
       'source_name', 'source_product', 'source_location', 'source_unit',
       'source_categories', 'edge_amount', 'edge_type'],
      dtype='object')

If you want to add or remove columns, you can pass in an iterable of formatting functions. These functions must satisfy the following rules:

* The take the keyword arguments `node`, `edge`, and `row`.
* They modify the dictionary `row` in place. Any return value is ignored.
* `node` and `edge` are dictionaries following the [wurst internal format](https://wurst.readthedocs.io/#internal-data-format). `node` is the target, and `edge` is both attributes of the edge and of the source.

Here is a simple example:

In [36]:
def remove_target_database(node, edge, row):
    del row['target_database']
    
def food_sector(node, edge, row):
    row['is_food'] = 'food' in edge['name'].lower()

In [37]:
df = bd.Database("US EEIO 1.1").edges_to_dataframe(formatters=[remove_target_database, food_sector])
df

Getting activity data


100%|███████████████████████| 2649/2649 [00:00<00:00, 325159.83it/s]


Adding exchange data to activities


100%|████████████████████| 162926/162926 [00:02<00:00, 63514.89it/s]


Filling out exchange data


100%|███████████████████████| 2649/2649 [00:00<00:00, 173898.32it/s]


Creating DataFrame
Compressing DataFrame


,target_id,target_code,target_name,target_reference_product,target_location,target_unit,target_type,source_id,source_database,source_code,source_name,source_product,source_location,source_unit,source_categories,edge_amount,edge_type,is_food
0,10,01624075-b520-3826-bd73-2068f7aa24e7,Frozen food; at manufacturer,NaN,United States,USD,process,468,US EEIO 1.1,26bed504-3f97-3a2b-aa83-ffbe94f3b371,Frozen food; at manufacturer,NaN,United States,,NaN,1.000000e+00,production,True
1,10,01624075-b520-3826-bd73-2068f7aa24e7,Frozen food; at manufacturer,NaN,United States,USD,process,446,US EEIO 1.1,1bafcbbb-dbe0-338d-b9c1-8c355426cbef,State and local government enterprises,NaN,United States,,NaN,1.338032e-03,technosphere,False
2,10,01624075-b520-3826-bd73-2068f7aa24e7,Frozen food; at manufacturer,NaN,United States,USD,process,1029,US EEIO 1.1,20185046-64bb-4c09-a8e7-e8a9e144ca98,Dinitrogen monoxide,NaN,NaN,,NaN,1.114413e-07,biosphere,False
3,10,01624075-b520-3826-bd73-2068f7aa24e7,Frozen food; at manufacturer,NaN,United States,USD,process,1737,US EEIO 1.1,7ae398b3-8532-11e0-9d78-0800200c9a66,ethylene glycol,NaN,NaN,,NaN,1.223946e-07,biosphere,False
4,10,01624075-b520-3826-bd73-2068f7aa24e7,Frozen food; at manufacturer,NaN,United States,USD,process,417,US EEIO 1.1,0bb0108f-c486-32b3-b059-e0c6c8380571,"Tobacco, cotton, sugarcane, peanuts, sugar bee...",NaN,United States,,NaN,9.990870e-06,technosphere,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162921,397,fe5971a7-2610-32ca-8193-a94873de130c,Automatic controls for HVAC and refrigeration ...,NaN,United States,USD,process,1435,US EEIO 1.1,5b2a19b9-1243-44ae-b76c-c0d92159d5d6,2-METHOXYETHANOL,NaN,NaN,,NaN,8.602175e-12,biosphere,False
162922,397,fe5971a7-2610-32ca-8193-a94873de130c,Automatic controls for HVAC and refrigeration ...,NaN,United States,USD,process,2631,US EEIO 1.1,fd7aa71c-508c-480d-81a6-8052aad92646,sulfur dioxide,NaN,NaN,,NaN,2.635425e-08,biosphere,False
162923,397,fe5971a7-2610-32ca-8193-a94873de130c,Automatic controls for HVAC and refrigeration ...,NaN,United States,USD,process,1471,US EEIO 1.1,5fd672a0-cb68-39e6-88dc-db1a9281c57b,"2,4,5,2',5'-PENTACHLOROBIPHENYL",NaN,NaN,,NaN,9.791193e-19,biosphere,False
162924,397,fe5971a7-2610-32ca-8193-a94873de130c,Automatic controls for HVAC and refrigeration ...,NaN,United States,USD,process,598,US EEIO 1.1,7e5c6ee1-a47e-3afd-9278-dcb5e1ee50b5,"Screws, nuts, and bolts; at manufacturer",NaN,United States,,NaN,1.810643e-02,technosphere,False


In the case of `target_name`, the dataframe has more than 150.000 rows, but only 388 unique values.

You can skip the conversion to categorical columns by passing `categorical=False`.

To save on memory, we turn some columns into categorical columns, where each unique value is only stored once.

In [38]:
df.dtypes

target_id                      int64
target_code                   object
target_name                 category
target_reference_product    category
target_location             category
target_unit                 category
target_type                 category
source_id                      int64
source_database             category
source_code                 category
source_name                 category
source_product              category
source_location             category
source_unit                 category
source_categories           category
edge_amount                  float64
edge_type                   category
is_food                         bool
dtype: object

In [39]:
df['target_name']

0                              Frozen food; at manufacturer
1                              Frozen food; at manufacturer
2                              Frozen food; at manufacturer
3                              Frozen food; at manufacturer
4                              Frozen food; at manufacturer
                                ...                        
162921    Automatic controls for HVAC and refrigeration ...
162922    Automatic controls for HVAC and refrigeration ...
162923    Automatic controls for HVAC and refrigeration ...
162924    Automatic controls for HVAC and refrigeration ...
162925    Automatic controls for HVAC and refrigeration ...
Name: target_name, Length: 162926, dtype: category
Categories (388, object): ['Abrasive products; at manufacturer', 'Accounting, tax preparation, bookkeeping, and..., 'Adhesives; at manufacturer', 'Advertising and public relations', ..., 'Wiring devices; at manufacturer', 'Wood kitchen cabinets and countertops; at man..., 'Wood pulp; at m

We can also get a dataframe of the edges for a specific node. Here we get all edges, but you can filter this further with the edge constructors `.production()`, `.technosphere()`, and `.biosphere()`.

In [40]:
df = activity.exchanges().to_dataframe()
df

,target_id,target_database,target_code,target_name,target_reference_product,target_location,target_unit,target_type,source_id,source_database,source_code,source_name,source_product,source_location,source_unit,source_categories,edge_amount,edge_type
0,268,US EEIO 1.1,a4d1f390-eb93-3e56-956b-6efb50b9d0bf,General merchandise stores,NaN,United States,USD,process,616,US EEIO 1.1,8f7034e4-99fe-324d-9fd9-08be3ee2f06a,General merchandise stores,NaN,United States,,Technosphere Flows::44-45: Retail Trade,1.000000e+00,production
1,268,US EEIO 1.1,a4d1f390-eb93-3e56-956b-6efb50b9d0bf,General merchandise stores,NaN,United States,USD,process,695,US EEIO 1.1,cc2b30ca-77b6-30d2-b8ac-a2377638b113,Light fixtures; at manufacturer,NaN,United States,,31-33: Manufacturing::3351: Electric Lighting ...,1.095284e-05,technosphere
2,268,US EEIO 1.1,a4d1f390-eb93-3e56-956b-6efb50b9d0bf,General merchandise stores,NaN,United States,USD,process,1086,US EEIO 1.1,28999907-a8a7-45b3-857e-836495ca2aa0,benzene,NaN,NaN,,air::unspecified,4.439622e-08,biosphere
3,268,US EEIO 1.1,a4d1f390-eb93-3e56-956b-6efb50b9d0bf,General merchandise stores,NaN,United States,USD,process,596,US EEIO 1.1,7d1b824e-99fb-3fc5-b351-85b0fc0c983a,Small electrical appliances; at manufacturer,NaN,United States,,31-33: Manufacturing::3352: Household Applianc...,7.721754e-04,technosphere
4,268,US EEIO 1.1,a4d1f390-eb93-3e56-956b-6efb50b9d0bf,General merchandise stores,NaN,United States,USD,process,731,US EEIO 1.1,dfd0ccab-61e3-34a8-9fce-316a77b2334e,Air transport,NaN,United States,,Technosphere Flows::48-49: Transportation and ...,1.040520e-03,technosphere
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,268,US EEIO 1.1,a4d1f390-eb93-3e56-956b-6efb50b9d0bf,General merchandise stores,NaN,United States,USD,process,1845,US EEIO 1.1,8aa9184d-fa90-4048-a84e-4d8e7a34aca1,"ethane, 1,2-dibromo-",NaN,NaN,,air::unspecified,2.281681e-14,biosphere
206,268,US EEIO 1.1,a4d1f390-eb93-3e56-956b-6efb50b9d0bf,General merchandise stores,NaN,United States,USD,process,1703,US EEIO 1.1,770c88e4-cd71-315c-b0b1-ea502618eb04,trichloroethylene,NaN,NaN,,air::unspecified,6.124519e-09,biosphere
207,268,US EEIO 1.1,a4d1f390-eb93-3e56-956b-6efb50b9d0bf,General merchandise stores,NaN,United States,USD,process,605,US EEIO 1.1,83e2a8d8-f4bc-3d20-8517-4c2d73119627,Computer terminals and other computer peripher...,NaN,United States,,31-33: Manufacturing::3341: Computer and Perip...,9.857558e-05,technosphere
208,268,US EEIO 1.1,a4d1f390-eb93-3e56-956b-6efb50b9d0bf,General merchandise stores,NaN,United States,USD,process,727,US EEIO 1.1,dd800655-77a3-3849-b9ad-e6e48775ef3d,Advertising and public relations,NaN,United States,,"54: Professional, Scientific, and Technical Se...",3.168657e-02,technosphere


Same columns as before.

In [41]:
df.columns

Index(['target_id', 'target_database', 'target_code', 'target_name',
       'target_reference_product', 'target_location', 'target_unit',
       'target_type', 'source_id', 'source_database', 'source_code',
       'source_name', 'source_product', 'source_location', 'source_unit',
       'source_categories', 'edge_amount', 'edge_type'],
      dtype='object')

We can also get dataframes for LCA calculation results.

In [42]:
lca = bc.LCA({product: 1}, method=('Impact Potential', 'HRSP'))
lca.lci()
lca.lcia()

By default, this method looks at the `characterized_inventory` matrix, and sorts by the top 200 values (using absolute value).

In [43]:
df = lca.to_dataframe()
df

,row_index,col_index,amount,row_id,col_id,row_database,row_code,row_name,row_location,row_unit,row_type,row_categories,row_product,col_database,col_code,col_name,col_location,col_unit,col_type,col_reference_product
0,257,130,8.585946e-04,1044,140,US EEIO 1.1,21e46cb8-6233-4c99-bac3-c41d2ab99498,"particulates, < 2.5 um",None,,emission,air::unspecified,None,US EEIO 1.1,4ebccf6f-bf79-33b8-be99-e37984272d17,"Tobacco, cotton, sugarcane, peanuts, sugar bee...",United States,USD,process,None
1,55,130,7.572715e-04,841,140,US EEIO 1.1,08a91e70-3ddc-11dd-91be-0050c2490048,"particulates, < 10 um",None,,emission,air::unspecified,None,US EEIO 1.1,4ebccf6f-bf79-33b8-be99-e37984272d17,"Tobacco, cotton, sugarcane, peanuts, sugar bee...",United States,USD,process,None
2,1843,130,1.248036e-07,2631,140,US EEIO 1.1,fd7aa71c-508c-480d-81a6-8052aad92646,sulfur dioxide,None,,emission,air::unspecified,None,US EEIO 1.1,4ebccf6f-bf79-33b8-be99-e37984272d17,"Tobacco, cotton, sugarcane, peanuts, sugar bee...",United States,USD,process,None
3,1043,130,1.729770e-05,1831,140,US EEIO 1.1,87883a4e-1e3e-4c9d-90c0-f1bea36f8014,ammonia,None,,emission,air::unspecified,None,US EEIO 1.1,4ebccf6f-bf79-33b8-be99-e37984272d17,"Tobacco, cotton, sugarcane, peanuts, sugar bee...",United States,USD,process,None
4,1445,130,1.122879e-06,2233,140,US EEIO 1.1,c1b91234-6f24-417b-8309-46111d09c457,nitrogen oxides,None,,emission,air::unspecified,None,US EEIO 1.1,4ebccf6f-bf79-33b8-be99-e37984272d17,"Tobacco, cotton, sugarcane, peanuts, sugar bee...",United States,USD,process,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,257,162,5.476811e-08,1044,172,US EEIO 1.1,21e46cb8-6233-4c99-bac3-c41d2ab99498,"particulates, < 2.5 um",None,,emission,air::unspecified,None,US EEIO 1.1,636743f5-f91b-3cf8-b3c7-f9f871498ee0,Compressed Gases; at manufacturer,United States,USD,process,None
196,257,221,4.919673e-08,1044,231,US EEIO 1.1,21e46cb8-6233-4c99-bac3-c41d2ab99498,"particulates, < 2.5 um",None,,emission,air::unspecified,None,US EEIO 1.1,8bc341ea-a073-3b64-a22e-6397582a2e2e,Animal farms and aquaculture ponds (except cat...,United States,USD,process,None
197,1043,221,1.581310e-06,1831,231,US EEIO 1.1,87883a4e-1e3e-4c9d-90c0-f1bea36f8014,ammonia,None,,emission,air::unspecified,None,US EEIO 1.1,8bc341ea-a073-3b64-a22e-6397582a2e2e,Animal farms and aquaculture ponds (except cat...,United States,USD,process,None
198,257,269,4.913991e-08,1044,279,US EEIO 1.1,21e46cb8-6233-4c99-bac3-c41d2ab99498,"particulates, < 2.5 um",None,,emission,air::unspecified,None,US EEIO 1.1,ad81ce2a-e3c5-3695-a2ef-812cd8b79dd3,Other plastic products; at manufacturer,United States,USD,process,None


The columns labels are a bit different, as we don't have target and source but instead matrix rows and columns. The meaning of these rows and columns changes from matrix to matrix. The same pattern with `'row_product'`, `'col_reference_product'`, and `'row_categories'` applies though.

In [44]:
df.columns

Index(['row_index', 'col_index', 'amount', 'row_id', 'col_id', 'row_database',
       'row_code', 'row_name', 'row_location', 'row_unit', 'row_type',
       'row_categories', 'row_product', 'col_database', 'col_code', 'col_name',
       'col_location', 'col_unit', 'col_type', 'col_reference_product'],
      dtype='object')

We can get dataframes for any matrix. In standard LCA, the matrices are:

* inventory
* technosphere_matrix
* biosphere_matrix
* characterization_matrix
* characterized_inventory

Regionalization adds more matrices. Note that for other matrices you will need to specify the row and column mapping dictionaries, see the docstring.

In [45]:
lca.to_dataframe(matrix_label='biosphere_matrix')

,row_index,col_index,amount,row_id,col_id,row_database,row_code,row_name,row_location,row_unit,row_type,row_categories,row_product,col_database,col_code,col_name,col_location,col_unit,col_type,col_reference_product
0,232,18,372.440552,1019,28,US EEIO 1.1,1ece2361-87e0-355c-a702-ff268570ca3e,Coal,None,,emission,resource::in ground,None,US EEIO 1.1,08f1c4b8-03f9-360c-87be-31ad6c778da5,Coal; at mine,United States,USD,process,None
1,947,18,0.174231,1735,28,US EEIO 1.1,7ae371aa-8532-11e0-9d78-0800200c9a66,Carbon dioxide,None,,emission,air::unspecified,None,US EEIO 1.1,08f1c4b8-03f9-360c-87be-31ad6c778da5,Coal; at mine,United States,USD,process,None
2,232,75,0.283722,1019,85,US EEIO 1.1,1ece2361-87e0-355c-a702-ff268570ca3e,Coal,None,,emission,resource::in ground,None,US EEIO 1.1,2bf3d179-abc9-3d6f-983f-f2de03471649,Other support activities for mining,United States,USD,process,None
3,947,75,0.116804,1735,85,US EEIO 1.1,7ae371aa-8532-11e0-9d78-0800200c9a66,Carbon dioxide,None,,emission,air::unspecified,None,US EEIO 1.1,2bf3d179-abc9-3d6f-983f-f2de03471649,Other support activities for mining,United States,USD,process,None
4,1388,95,96.912102,2176,105,US EEIO 1.1,b91d0527-9a01-4a86-b420-c62b70629ba4,"Occupation, forest",None,,emission,resource::land,None,US EEIO 1.1,392eb1e3-3cd1-34c7-948c-c177114e8d20,Timber and raw forest products; at forest,United States,USD,process,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,947,121,0.092894,1735,131,US EEIO 1.1,7ae371aa-8532-11e0-9d78-0800200c9a66,Carbon dioxide,None,,emission,air::unspecified,None,US EEIO 1.1,48b2f105-8ae0-36e2-ad0a-cf7101ab8f4c,Residential building repair and maintanence,United States,USD,process,None
196,947,39,0.092228,1735,49,US EEIO 1.1,7ae371aa-8532-11e0-9d78-0800200c9a66,Carbon dioxide,None,,emission,air::unspecified,None,US EEIO 1.1,15615fdc-2456-3e6c-bd24-9ed9a13d2599,Health care buildings,United States,USD,process,None
197,947,191,0.090496,1735,201,US EEIO 1.1,7ae371aa-8532-11e0-9d78-0800200c9a66,Carbon dioxide,None,,emission,air::unspecified,None,US EEIO 1.1,7452907b-74cc-3106-aaaa-5560c0645af6,Flours and malts; at manufacturer,United States,USD,process,None
198,947,340,0.090194,1735,350,US EEIO 1.1,7ae371aa-8532-11e0-9d78-0800200c9a66,Carbon dioxide,None,,emission,air::unspecified,None,US EEIO 1.1,d920c723-5594-34a5-8b55-0cbe517c2f9f,Polystyrene foam products; at manufacturer,United States,USD,process,None


# bw2calc

## Specify `data_objs` and new functional unit

The biggest change in Brightway 2.5 is the use of [bw_processing](https://github.com/brightway-lca/bw_processing) and [matrix_utils](https://github.com/brightway-lca/matrix_utils), which in turn lead to a new calling convention for LCA class instantiation. These new libraries allow for calculations to be shipped off to cloud servers, which means that they have no connection to `bw2data` or metadata about the meaning of any ID values - instead, they only solve linear problems.

The new calling convention is therefore functional unit **with the IDs for the nodes** and **datapackages as `data_objs`**. Here is an example:

In [46]:
product = next(node for node in bd.Database("US EEIO 1.1") if node['type'] == 'product')
product

'Rubber tires; at manufacturer' (, United States, ('31-33: Manufacturing', '3262: Rubber Product Manufacturing'))

In [47]:
database_dp = bd.Database("US EEIO 1.1").datapackage()
lcia_dp = bd.Method(('Impact Potential', 'HC')).datapackage()

In [48]:
lca = bc.LCA({product.id: 1}, data_objs=[database_dp, lcia_dp])
lca.lci()
lca.lcia()

In [49]:
lca.score

1.6568481521071192e-10

The old calling convention will still work, but only if you have `bw2data` installed, the correct project selected, etc.

## `bw2data.prepare_lca_inputs`

If you don't want to remember the new calling convention, you can use a helper function: `bw2data.prepare_lca_inputs`. It will return three things: A new demand dictionary, the datapackages, and (if `remapping=True`, the default) dictionaries to allow you to map matrix indices back to Brightway (database, code) keys.

In [50]:
fu, dps, remapping = bd.prepare_lca_inputs({bd.get_node(name='Steel'): 1}, ('IPCC', 'simple'))

In [51]:
fu

{4: 1}

In [52]:
dps

In [53]:
remapping

{'activity': {8: ('Mobility example', '6f233d5e001dc3ea41b1a53bbb815521'),
  3: ('Mobility example', '7fabbb483617d7a9325f7dee36a11562'),
  9: ('Mobility example', 'CO2'),
  7: ('Mobility example', 'Driving an combustion car'),
  1: ('Mobility example', 'Driving an electric car'),
  5: ('Mobility example', 'Electricity'),
  6: ('Mobility example', 'Lithium'),
  4: ('Mobility example', 'Steel'),
  2: ('Mobility example', 'edb43eb8b3e187759b38c55ae4228ffc')},
 'product': {8: ('Mobility example', '6f233d5e001dc3ea41b1a53bbb815521'),
  3: ('Mobility example', '7fabbb483617d7a9325f7dee36a11562'),
  9: ('Mobility example', 'CO2'),
  7: ('Mobility example', 'Driving an combustion car'),
  1: ('Mobility example', 'Driving an electric car'),
  5: ('Mobility example', 'Electricity'),
  6: ('Mobility example', 'Lithium'),
  4: ('Mobility example', 'Steel'),
  2: ('Mobility example', 'edb43eb8b3e187759b38c55ae4228ffc')},
 'biosphere': {8: ('Mobility example', '6f233d5e001dc3ea41b1a53bbb815521'),
 

## No automatic remapping

Previously, `bw2calc.LCA` would automatically change the integer values given in `bw2data.mapping` to keys. This is no longer the case, as we assume normal behaviour in the future is to prefer node IDs to keys, and also don't have any guarantees on whether `bw2data` is available. Therefore, **you need to call LCA.remap_inventory_dicts() manually**.

In [54]:
lca = bc.LCA(demand=fu, data_objs=dps, remapping_dicts=remapping)
lca.lci()
lca.lcia()
lca.demand

{4: 1}

In [55]:
lca.dicts.product[bd.get_node(name='Steel').id]

3

In [56]:
lca.remap_inventory_dicts()

In [57]:
lca.demand

{('Mobility example', 'Steel'): 1}

In [58]:
lca.dicts.product[bd.get_node(name='Steel').key]

3

In [59]:
list(bd.Database("Mobility example"))

['Electricity' (kilowatt hour, GLO, None),
 'Steel' (kilogram, GLO, None),
 'Electric car' (unit, GLO, ('Electric car',)),
 'Combustion car' (unit, GLO, ('Combustion car',)),
 'Driving an combustion car' (kilometer, Europe, None),
 'Lithium' (kilogram, GLO, None),
 'Electric car battery' (unit, GLO, ('Electric car battery',)),
 'Driving an electric car' (kilometer, Europe, None),
 'CO2' (kilogram, None, None)]

## `.redo_lci` ➡️ `.lci`, `.redo_lcia` ➡️ `.lcia`

The preferred way to do additional LCI or LCIA calculations has changed, and has been simplified to the same way we do an initial caluculation, namely with `.lci(new_demand_dict)` and `.lcia(new_demand_dict)`.

In [60]:
lca = bc.LCA(demand=fu, data_objs=dps)
lca.lci()
lca.lcia()

for act in [
    bd.get_node(name='Driving an combustion car'),
    bd.get_node(name='Driving an electric car'), 
]:
    lca.lcia({act.id: 1})
    print(lca.score, act)

0.15490749959371897 'Driving an combustion car' (kilometer, Europe, None)
0.18248161259647167 'Driving an electric car' (kilometer, Europe, None)


Note the following **important condition**: The functional unit must be specified either as integer IDs (see above), or as Brightway (database, code) keys *if you have run `remap_inventory_dicts()`*. The software will help you as much as possible:

In [61]:
lca.lcia({bd.get_node(name='Driving an combustion car').key: 1})

KeyError: "Key '('Mobility example', 'Driving an combustion car')' not in product dictionary; make sure to pass the integer id, not a key like `('foo', 'bar')` or an `Actiivity` or `Node` object."

## `LCA` object can now do Monte Carlo

There is now no longer a separate Monte Carlo class, as the sampling of values has moved to `matrix_utils` as is more generic than before - we don't need to assume that people only have probability distribution functions, but are open to many more types of quantitative uncertainty or scenarios.

Therefore, calls to `MonteCarloLCA` need to **change to `LCA(..., use_distributions=True)`**, and you can use the normal `.lci()` and `.lcia()` functions.

In [ ]:
ipcc = bd.Method(('IPCC', 'uncertain'))
ipcc.register()
ipcc.write([(('Mobility example', 'CO2'), {'uncertainty_type': 3, 'amount': 1, 'loc': 1, 'scale': 0.1})])

In [ ]:
lca = bc.LCA(
    demand={bd.get_node(name='Driving an electric car'): 1}, 
    method=('IPCC', 'uncertain'),
    use_distributions=True
)
lca.lci()
lca.lcia()

for _ in zip(range(10), lca):
    print(lca.score)

### `keep_first_iteration`

Sometimes you want to keep the values sampled when you set up the LCA object, instead of getting the next value in a sequence when you start iterating over Monte Carlo results. In this case, just call `.keep_first_iteration` and it will skip the first iteration step.

In [ ]:
lca = bc.LCA(
    demand={bd.get_node(name='Driving an electric car'): 1}, 
    method=('IPCC', 'uncertain'),
    use_distributions=True
)
lca.lci()
lca.lcia()
lca.keep_first_iteration()
print("Score after setup:", lca.score)

for _ in zip(range(4), lca):
    print("In loop:", lca.score)

## New `.dicts` accessor

The old methods still work (well, they will in the next dev release :), but the preferred way is to do the following:

In [ ]:
product = next(node for node in bd.Database("US EEIO 1.1") if node['type'] == 'product')
activity = next(node for node in bd.Database("US EEIO 1.1") if node['type'] == 'process')
emission = next(node for node in bd.Database("US EEIO 1.1") if node['type'] == 'emission')

In [ ]:
lca = bc.LCA({product: 1})
lca.lci()

To matrix index:

In [ ]:
act_matrix = lca.dicts.activity[activity.id]
act_matrix

To database ID:

In [ ]:
lca.dicts.activity.reversed[act_matrix], activity.id

And the same for products and biosphere flows:

In [ ]:
lca.dicts.product[product.id], lca.dicts.biosphere[emission.id]